In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import (api_key, gkey)

### Store Part I results into DataFrame

In [2]:
# Import saved CSV file into a new dataframe
cities_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,77.36,67,14,17.63,PF,1627522010
1,George Town,5.4112,100.3354,79.90,90,20,2.30,MY,1627522011
2,Mataura,-46.1927,168.8643,57.47,72,55,4.94,NZ,1627521773
3,Lebu,-37.6167,-73.6500,49.24,79,0,8.01,CL,1627522011
4,Pangkalanbuun,-2.6833,111.6167,82.04,76,30,4.85,ID,1627522012
...,...,...,...,...,...,...,...,...,...
555,Sayansk,54.1088,102.1648,57.79,81,100,13.38,RU,1627522176
556,Kupang,-10.1667,123.5833,83.53,58,0,11.68,ID,1627522177
557,Paralímni,35.0375,33.9833,87.12,42,0,8.01,CY,1627522177
558,Beringovskiy,63.0500,179.3167,48.16,72,100,16.91,RU,1627522177


### Humidity Heatmap

In [3]:
# Define Latitudes and Longitudes as locations for heatmap
locations = cities_weather_df[["Lat", "Lng"]].astype(float)

# Define Humidity as weights for heatmap
humidity = cities_weather_df["Humidity"].astype(float)

In [4]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Create and empty figure with Google maps
fig = gmaps.figure(center=(30,0), zoom_level=2)

# Create heatmap_layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                    max_intensity=100, dissipating=False, point_radius=3)

# Add heatmap_layer
fig.add_layer(heatmap_layer)

# Display map with heat layer
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
# Generate Dataframe for cities with ideal weather conditions and dropping rows with null values
hotel_df = (cities_weather_df.loc[((cities_weather_df["Max Temp"] < 75.00) & 
                                           (cities_weather_df["Max Temp"] > 65.00)) & 
                                           (cities_weather_df["Wind Speed"] < 10.00) & 
                                           (cities_weather_df["Cloudiness"] == 0)]).dropna()

# Add new column Hotel Name to store first hotel's name
hotel_df["Hotel Name"] = ""

# Display hotel dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
98,Pilar,-9.5972,-35.9567,67.93,92,0,1.19,BR,1627522040,
147,Tekeli,44.8300,78.8239,68.85,32,0,3.53,KZ,1627522052,
168,Trebisacce,39.8708,16.5312,74.53,32,0,6.38,IT,1627522058,
185,Bukama,-9.2000,25.8500,66.16,43,0,3.42,CD,1627522063,
190,Broome,42.2506,-75.8330,65.39,86,0,2.39,US,1627522064,
235,Morros,-9.4467,-46.3003,74.41,35,0,3.69,BR,1627521917,
238,Medina del Campo,41.3124,-4.9141,65.14,68,0,6.55,ES,1627522078,
249,Mizdah,31.4451,12.9801,69.85,58,0,6.55,LY,1627522081,
323,São José da Coroa Grande,-8.8978,-35.1478,73.98,85,0,4.72,BR,1627522104,
402,Petro-Slavyanka,59.8022,30.5069,66.27,90,0,4.47,RU,1627522129,


### Hotel Map

In [7]:
# Base url for Google Places nearbyserach API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set up parameter directory
params = {"radius": 5000, 
          "type":"lodging", 
          "key":api_key,}

# For loop to update location parameter
# Using Google Places API and storing first hotel in a new column of the dataframe
for rows, values in hotel_df.iterrows():
    
    params["location"] = f"{values[1]},{values[2]}"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[rows,'Hotel Name'] = response['results'][0]['name']
        print(f"The first hotel near to {values[0]} is {hotel_df.loc[rows,'Hotel Name']}")        
    except(KeyError, IndexError):
        print("No hotel found")
    print("----------------")

The first hotel near to Pilar is Pousada e churrascaria Manguaba
----------------
The first hotel near to Tekeli is Vuslat Tekeli
----------------
The first hotel near to Trebisacce is Hotel Ristorante Pizzeria 111
----------------
No hotel found
----------------
The first hotel near to Broome is Chenango Valley State Park
----------------
No hotel found
----------------
The first hotel near to Medina del Campo is Hotel Villa de Ferias
----------------
The first hotel near to Mizdah is محمدالمبروك الاريقط
----------------
The first hotel near to São José da Coroa Grande is Crôa Mares Hotel
----------------
The first hotel near to Petro-Slavyanka is Izhora
----------------
The first hotel near to Nyurba is Gostinitsa Alrosa
----------------
The first hotel near to Riachão das Neves is Casa Costa
----------------
The first hotel near to Yulara is Desert Gardens Hotel - Ayers Rock Resort
----------------
The first hotel near to Tamandaré is Pousada 100 Passos
----------------


In [8]:
# Display Hotel dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
98,Pilar,-9.5972,-35.9567,67.93,92,0,1.19,BR,1627522040,Pousada e churrascaria Manguaba
147,Tekeli,44.8300,78.8239,68.85,32,0,3.53,KZ,1627522052,Vuslat Tekeli
168,Trebisacce,39.8708,16.5312,74.53,32,0,6.38,IT,1627522058,Hotel Ristorante Pizzeria 111
185,Bukama,-9.2000,25.8500,66.16,43,0,3.42,CD,1627522063,
190,Broome,42.2506,-75.8330,65.39,86,0,2.39,US,1627522064,Chenango Valley State Park
235,Morros,-9.4467,-46.3003,74.41,35,0,3.69,BR,1627521917,
238,Medina del Campo,41.3124,-4.9141,65.14,68,0,6.55,ES,1627522078,Hotel Villa de Ferias
249,Mizdah,31.4451,12.9801,69.85,58,0,6.55,LY,1627522081,محمدالمبروك الاريقط
323,São José da Coroa Grande,-8.8978,-35.1478,73.98,85,0,4.72,BR,1627522104,Crôa Mares Hotel
402,Petro-Slavyanka,59.8022,30.5069,66.27,90,0,4.47,RU,1627522129,Izhora


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_marker = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations_marker, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))